<a href="https://colab.research.google.com/github/smokingelephants/RL_test/blob/master/saving_loading_a2c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Baselines, a Fork of OpenAI Baselines - Training, Saving and Loading

Github Repo: [https://github.com/hill-a/stable-baselines](https://github.com/hill-a/stable-baselines)

Medium article: [https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82](https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82)

## Install Dependencies and Stable Baselines Using Pip

List of full dependencies can be found in the [README](https://github.com/hill-a/stable-baselines).

```

sudo apt-get update && sudo apt-get install cmake libopenmpi-dev zlib1g-dev
```


```

pip install stable-baselines
```

In [2]:
!apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines==2.2.0 box2d box2d-kengz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.10.2-1ubuntu2).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libopenmpi-dev is already the newest version (2.1.1-8).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 6 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fe

## Import policy, RL agent, ...

In [0]:
import gym
import numpy as np

from stable_baselines.common.policies import MlpPolicy, CnnPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C,DQN
from stable_baselines.deepq.policies import DQNPolicy


## Create the Gym env and instantiate the agent

For this example, we will use Lunar Lander environment.

"Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt. Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine. "

Lunar Lander environment: [https://gym.openai.com/envs/LunarLander-v2/](https://gym.openai.com/envs/LunarLander-v2/)



Note: vectorized environments allow to easily multiprocess training. In this example, we are using only one process, hence the DummyVecEnv.

We chose the MlpPolicy because input of CartPole is a feature vector, not images.

The type of action to use (discrete/continuous) will be automatically deduced from the environment action space



In [0]:
#env = gym.make('LunarLander-v2')
env = gym.make('CartPole-v0')
#env = gym.make('Pong-v0')
# vectorized environments allow to easily multiprocess training
# we demonstrate its usefulness in the next examples
#env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run
import pdb;pdb.set_trace()
model = DQN(DQNPolicy, env)
#model = A2C(CnnPolicy, env, ent_coef=0.1, verbose=0)


--Return--
> <ipython-input-81-085ccda4f0df>(6)<module>()->None
-> import pdb;pdb.set_trace()
--KeyboardInterrupt--
(Pdb) l
  1  	env = gym.make('CartPole-v0')
  2  	#env = gym.make('Pong-v0')
  3  	# vectorized environments allow to easily multiprocess training
  4  	# we demonstrate its usefulness in the next examples
  5  	#env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run
  6  ->	import pdb;pdb.set_trace()
  7  	model = DQN(DQNPolicy, env)
  8  	#model = A2C(CnnPolicy, env, ent_coef=0.1, verbose=0)
[EOF]
(Pdb) s
> /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py(2885)run_code()
-> sys.excepthook = old_excepthook
(Pdb) s
> /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py(2901)run_code()
-> outflag = 0
(Pdb) s
> /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py(2902)run_code()
-> return outflag
(Pdb) s
--Return--
> /usr/local/lib/python3.6/dist-packages/IPython/core/interactiv

We create a helper function to evaluate the agent:

In [0]:
def evaluate(model, num_steps=1000):
  """
  Evaluate a RL agent
  :param model: (BaseRLModel object) the RL Agent
  :param num_steps: (int) number of timesteps to evaluate it
  :return: (float) Mean reward for the last 100 episodes
  """
  episode_rewards = [0.0]
  obs = env.reset()
  for i in range(num_steps):
      # _states are only useful when using LSTM policies
      bose_shape=obs.shape
      action, _states = model.predict(obs)
      # here, action, rewards and dones are arrays
      # because we are using vectorized env
      obs, rewards, dones, info = env.step(action)
      
      # Stats
      episode_rewards[-1] += rewards[0]
      if dones[0]:
          obs = env.reset()
          episode_rewards.append(0.0)
  # Compute mean reward for the last 100 episodes
  mean_100ep_reward = round(np.mean(episode_rewards[-100:]), 1)
  print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards))
  print(bose_shape)
  print(obs)
  return mean_100ep_reward

Let's evaluate the un-trained agent, this should be a random agent.

In [64]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_steps=10000)

Mean reward: -191.4 Num episodes: 104
(1, 8)
[[ 0.00203476  0.6566262  -0.15301418 -1.0697302   0.4138869   0.35781956
   0.          0.        ]]


## Train the agent and save it

Warning: this may take a while

In [0]:
# Train the agent
model.learn(total_timesteps=50000)
# Save the agent
model.save("a2c_lunar")
#del model  # delete trained model to demonstrate loading

## Load the trained agent

In [44]:
print('hello')

hello


In [50]:
model = A2C.load("a2c_lunar")

Loading a model without an environment, this model cannot be trained until it has a valid environment.


In [68]:
# Evaluate the trained agent
mean_reward = evaluate(model, num_steps=10000)

Mean reward: -50.9 Num episodes: 39
(1, 8)
[[-0.04557629  0.5396952   0.09012793 -0.23574258 -0.1658564   0.03524155
   0.          0.        ]]
